In [ ]:
import pickle

file_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_Os.pickle"

with open(file_path, "rb") as f:
    data = pickle.load(f)

print(f"Loaded {len(data)} items from {file_path}")
first_item = data[0]
if isinstance(first_item, tuple):
    print("Tuple 長度:", len(first_item))
    for i, v in enumerate(first_item):
        print(f"  元素 {i} 型別:", type(v))
elif isinstance(first_item, dict):
    print("Keys:", first_item.keys())




Loaded 66607 items from /home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_Os.pickle
Tuple 長度: 3
  元素 0 型別: <class 'numpy.ndarray'>
  元素 1 型別: <class 'numpy.ndarray'>
  元素 2 型別: <class 'int'>


In [ ]:
import pickle
import torch
import numpy as np
from tqdm import tqdm

# === 載入資料 ===
path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector.pickle"
with open(path, "rb") as f:
    data = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 定義 similarity 函式 ===
def similarity_score(x1, x2):
    dist = torch.norm(x1 - x2, dim=1)  
    return 1 / (1 + dist)             

# === 批次計算平均 similarity ===
scores = []
for v1, v2, _ in tqdm(data, desc="計算中"):
    t1 = torch.tensor(v1, dtype=torch.float32, device=device).unsqueeze(0)
    t2 = torch.tensor(v2, dtype=torch.float32, device=device).unsqueeze(0)

    score = similarity_score(t1, t2)
    scores.append(score.item())

avg_score = np.mean(scores)
print(f"平均 similarity score: {avg_score:.4f}")


計算中: 100%|██████████| 57494/57494 [00:02<00:00, 23177.31it/s]

平均 similarity score: 0.4903


In [17]:
path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector.pickle"
with open(path, "rb") as f:
    data = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

check = data[21]

v1, v2, label = check
print("Label:", label)
t1 = torch.tensor(v1, dtype=torch.float32, device=device).unsqueeze(0)
t2 = torch.tensor(v2, dtype=torch.float32, device=device).unsqueeze(0)
check_score = similarity_score(t1, t2)
print(f"Similarity score: {check_score.item():.4f}")

Label: 1
Similarity score: 1.0000


In [4]:
import pickle
import torch
import numpy as np
import random
from tqdm import tqdm

# === 載入資料 ===
input_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_mix.pickle"
output_path = "/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_posneg_mixed.pickle"

with open(input_path, "rb") as f:
    data = pickle.load(f)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === similarity 函式 ===
def similarity_score(x1, x2):
    dist = torch.norm(x1 - x2, dim=1)  # L2 distance
    return 1 / (1 + dist)

# === 計算所有 similarity score ===
scored_data = []
for v1, v2, label in tqdm(data, desc="計算相似度"):
    t1 = torch.tensor(v1, dtype=torch.float32, device=device).unsqueeze(0)
    t2 = torch.tensor(v2, dtype=torch.float32, device=device).unsqueeze(0)
    score = similarity_score(t1, t2).item()
    scored_data.append((v1, v2, label, score))

# === 依照相似度排序 ===
scored_data.sort(key=lambda x: x[3], reverse=True)

# === 分成兩半 ===
half = len(scored_data) // 2
positive_samples = [(v1, v2, 1) for v1, v2, _, _ in scored_data[:half]]
negative_candidates = [ (v1, v2) for v1, v2, _, _ in scored_data[half:] ]

# === 打亂後配對建立負樣本 ===
all_vec1 = [v1 for v1, _ in negative_candidates]
all_vec2 = [v2 for _, v2 in negative_candidates]
random.shuffle(all_vec2)

negative_samples = [(v1, v2, 0) for v1, v2 in zip(all_vec1, all_vec2)]

# === 合併正負樣本 ===
final_dataset = positive_samples + negative_samples
random.shuffle(final_dataset)

# === 儲存 ===
with open(output_path, "wb") as f:
    pickle.dump(final_dataset, f)

print(f"[INFO] 原始樣本數: {len(data)}")
print(f"[INFO] 新資料集樣本數: {len(final_dataset)}")
print(f"[INFO] 已儲存至: {output_path}")


計算相似度:   0%|          | 0/57494 [00:00<?, ?it/s]

計算相似度: 100%|██████████| 57494/57494 [00:02<00:00, 23368.15it/s]


[INFO] 原始樣本數: 57494
[INFO] 新資料集樣本數: 57494
[INFO] 已儲存至: /home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_posneg_mixed.pickle


In [ ]:
#check label 1 and label 0 simiarity score
import pickle 

with open("/home/tommy/Projects/pcodeFcg/dataset/alignment_vector/train_arm_vector_posneg_mixed.pickle", "rb") as f:
    data = pickle.load(f)

#check label distribution
from collections import Counter
labels = [label for _, _, label in data]
label_counts = Counter(labels)
print("Label distribution:", label_counts)

for v1, v2, label in data:
    if label == 1:
        t1 = torch.tensor(v1, dtype=torch.float32, device=device).unsqueeze(0)
        t2 = torch.tensor(v2, dtype=torch.float32, device=device).unsqueeze(0)
        pos_score = similarity_score(t1, t2)
        print(f"Positive pair similarity score: {pos_score.item():.4f}")
        break
for v1, v2, label in data:
    if label == 0:
        t1 = torch.tensor(v1, dtype=torch.float32, device=device).unsqueeze(0)
        t2 = torch.tensor(v2, dtype=torch.float32, device=device).unsqueeze(0)
        neg_score = similarity_score(t1, t2)
        print(f"Negative pair similarity score: {neg_score.item():.4f}")
        break

    

Label distribution: Counter({0: 33304, 1: 33303})
Positive pair similarity score: 0.6704
Negative pair similarity score: 0.1096
